# Supervisor (Polyglot)

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.2.2/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client.core/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.Core.dll"
#r @"../../../../../../../.nuget/packages/fsharp.json/0.4.1/lib/netstandard2.0/FSharp.Json.dll"

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/AsyncSeq.fs
#!import ../../lib/fsharp/Runtime.fs
#!import ../../lib/fsharp/FileSystem.fs

In [ ]:
#if !INTERACTIVE
open Lib
#endif

In [ ]:
open Common
open SpiralFileSystem.Operators
open Microsoft.AspNetCore.SignalR.Client

## sendJson

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = SpiralNetworking.test_port_open port
    if portOpen then
        try
            let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{port}").Build()
            do! connection.StartAsync () |> Async.AwaitTask
            let! result = connection.InvokeAsync<string>("ClientToServerMsg", json) |> Async.AwaitTask
            do! connection.StopAsync () |> Async.AwaitTask
            trace Verbose (fun () -> $"sendJson / port: {port} / json: {json |> SpiralSm.ellipsis_end 200} / result: {result |> Option.ofObj |> Option.map (SpiralSm.ellipsis_end 200)}") _locals
            return Some result
        with ex ->
            trace Critical (fun () -> $"sendJson / port: {port} / json: {json |> SpiralSm.ellipsis_end 200} / ex: {ex |> SpiralSm.format_exception}") _locals
            return None
    else
        trace Debug (fun () -> "sendJson / error: port not open") _locals
        return None
}

## sendObj

In [ ]:
let inline sendObj port obj =
    obj
    |> System.Text.Json.JsonSerializer.Serialize
    |> sendJson port

## awaitCompiler

In [ ]:
type VSCPos = {| line : int; character : int |}
type VSCRange = VSCPos * VSCPos
type RString = VSCRange * string
type TracedError = {| trace : string list; message : string |}
type ClientErrorsRes =
    | FatalError of string
    | TracedError of TracedError
    | PackageErrors of {| uri : string; errors : RString list |}
    | TokenizerErrors of {| uri : string; errors : RString list |}
    | ParserErrors of {| uri : string; errors : RString list |}
    | TypeErrors of {| uri : string; errors : RString list |}

In [ ]:
let repositoryRoot = SpiralFileSystem.get_repository_root ()

In [ ]:
let inline awaitCompiler port cancellationToken = async {
    let host = "127.0.0.1"
    let struct (ct, disposable) = cancellationToken |> SpiralThreading.new_disposable_token
    let! ct = ct |> SpiralAsync.merge_cancellation_token_with_default_async

    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = SpiralNetworking.get_available_port (Some 60) port
        if availablePort <> port then
            inbox.Post (port, false)
        else
            let compilerPath =
                repositoryRoot </> "deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> System.IO.Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"

            let! exitCode, result =
                SpiralRuntime.execution_options (fun x ->
                    { x with
                        l0 = Some ct
                        l1 = $@"dotnet ""{dllPath}"" --port {availablePort} --default-int i32 --default-float f64"
                        l3 = Some (fun struct (_, line, _) -> async {
                            if line |> SpiralSm.contains $"System.IO.IOException: Failed to bind to address http://{host}:{port}: address already in use." then
                                inbox.Post (port, false)

                            if line |> SpiralSm.contains $"Server bound to: http://localhost:{availablePort}" then
                                do!
                                    SpiralNetworking.wait_for_port_access (Some 5000) true availablePort
                                    |> Async.Ignore

                                let rec loop retry = async {
                                    let _locals () = $"port: {availablePort} / retry: {retry} / {_locals ()}"
                                    try
                                        let pingObj = {| Ping = true |}
                                        let! pingResult = pingObj |> sendObj availablePort
                                        trace Verbose (fun () -> $"awaitCompiler / Ping / result: '{pingResult}'") _locals
                                    with ex ->
                                        trace Verbose (fun () -> $"awaitCompiler / Ping / ex: {ex |> SpiralSm.format_exception}") _locals
                                        do! Async.Sleep 10
                                        do! loop (retry + 1)
                                }
                                do! loop 0
                                inbox.Post (availablePort, true)
                        })
                        l4 = Some repositoryRoot
                    }
                )
                |> SpiralRuntime.execute_with_options_async

            trace Debug (fun () -> $"awaitCompiler / exitCode: {exitCode} / result: {result}") _locals
            disposable.Dispose ()
    }, ct)

    let! serverPort, managed = compiler.Receive ()

    let connection = HubConnectionBuilder().WithUrl($"http://{host}:{serverPort}").Build ()
    do! connection.StartAsync () |> Async.AwaitTask

    let event = Event<_> ()
    let disposable' = connection.On<string> ("ServerToClientMsg", event.Trigger)
    let stream =
        FSharp.Control.AsyncSeq.unfoldAsync
            (fun () -> async {
                let! msg = event.Publish |> Async.AwaitEvent
                return Some (msg |> FSharp.Json.Json.deserialize<ClientErrorsRes>, ())
            })
            ()

    let disposable' =
        new_disposable (fun () ->
            async {
                disposable'.Dispose ()
                do! connection.StopAsync () |> Async.AwaitTask
                disposable.Dispose ()
                if managed
                then do! SpiralNetworking.wait_for_port_access (Some 3000) false serverPort |> Async.Ignore
            }
            |> Async.RunSynchronously
        )

    return
        serverPort,
        stream,
        ct,
        disposable'
}

## getFilePathFromUri

In [ ]:
let inline getFilePathFromUri uri =
    match System.Uri.TryCreate (uri, System.UriKind.Absolute) with
    | true, uri -> uri.AbsolutePath |> System.IO.Path.GetFullPath
    | _ -> failwith "invalid uri"

## getCompilerPort

In [ ]:
let inline getCompilerPort () =
    13805

## serialize_obj

In [ ]:
    let serializeObj obj =
        obj
        |> FSharp.Json.Json.serialize
        |> SpiralSm.replace "\\\\" "\\"
        |> SpiralSm.replace "\\r\\n" "\n"
        |> SpiralSm.replace "\\n" "\n"

## buildFile

In [ ]:
let inline buildFile timeout port cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> SpiralFileSystem.read_all_text_async

    let stream, disposable = fileDir |> FileSystem.watchDirectory (fun _ -> false)
    use _ = disposable

    let struct (token, disposable) = SpiralThreading.new_disposable_token cancellationToken
    use _ = disposable

    let! serverPort, errors, ct, disposable = awaitCompiler port (Some token)
    use _ = disposable

    let fsxContentSeq =
        stream
        |> FSharp.Control.AsyncSeq.chooseAsync (function
            | _, (FileSystem.FileSystemChange.Changed (path, _))
                when (path |> System.IO.Path.GetFileName) = $"{fileName}.fsx"
                ->
                    fileDir </> path |> SpiralFileSystem.read_all_text_retry_async
            | _ -> None |> Async.init
        )
        |> FSharp.Control.AsyncSeq.map (fun content ->
            Some (content |> SpiralSm.replace "\r\n" "\n"), None
        )

    let inline printErrorData (data : {| uri : string; errors : RString list |}) =
        let fileName = data.uri |> System.IO.Path.GetFileName
        let errors =
            data.errors
            |> List.map snd
            |> SpiralSm.concat "\n"
        $"{fileName}:\n{errors}"

    let errorsSeq =
        errors
        |> FSharp.Control.AsyncSeq.choose (fun error ->
            match error with
            | FatalError message ->
                Some (message, error)
            | TracedError data ->
                Some (data.message, error)
            | PackageErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | TokenizerErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | ParserErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | TypeErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun (message, error) ->
            None, Some (message, error)
        )

    let timerSeq =
        1000
        |> FSharp.Control.AsyncSeq.intervalMs
        |> FSharp.Control.AsyncSeq.map (fun _ -> None, None)

    let outputSeq =
        [ fsxContentSeq; errorsSeq; timerSeq ]
        |> FSharp.Control.AsyncSeq.mergeAll

    let! outputChild =
        ((None, [], 0), outputSeq)
        ||> FSharp.Control.AsyncSeq.scan (
            fun (fsxContentResult, errors, typeErrorCount) (fsxContent, error) ->
                match fsxContent, error with
                | Some fsxContent, None -> Some fsxContent, errors, typeErrorCount
                | None, Some (_, FatalError "File main has a type error somewhere in its path.") ->
                    fsxContentResult, errors, typeErrorCount + 1
                | None, Some error -> fsxContentResult, error :: errors, typeErrorCount
                | None, None when typeErrorCount >= 1 ->
                    fsxContentResult, errors, typeErrorCount + 1
                | _ -> fsxContentResult, errors, typeErrorCount
        )
        |> FSharp.Control.AsyncSeq.takeWhileInclusive (fun (fsxContent, errors, typeErrorCount) ->
            trace Debug (fun () -> $"buildFile / takeWhileInclusive / path: {path} / fsxContent: {fsxContent |> Option.defaultValue System.String.Empty |> SpiralSm.ellipsis_end 400} / errors: {errors |> serializeObj} / typeErrorCount: {typeErrorCount}") _locals
#if INTERACTIVE
            let errorWait = 2
#else
            let errorWait = 4
#endif
            match fsxContent, errors with
            | None, [] when typeErrorCount > errorWait -> false
            | None, [] -> true
            | _ -> false
        )
        |> FSharp.Control.AsyncSeq.tryLast
        |> Async.withCancellationToken ct
        |> Async.catch
        |> Async.runWithTimeoutAsync timeout
        |> Async.StartChild

    let fullPathUri = fullPath |> SpiralFileSystem.normalize_path |> SpiralFileSystem.new_file_uri

    let fileOpenObj = {| FileOpen = {| uri = fullPathUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    // do! Async.Sleep 60

    let buildFileObj = {| BuildFile = {| uri = fullPathUri; backend = "Fsharp" |} |}
    let! _buildFileResult = buildFileObj |> sendObj serverPort

    let! result =
        outputChild
        |> Async.map (function
            | Some (Ok (Some (message, errors, _))) ->
                message, errors |> List.distinct |> List.rev
            | Some (Error ex) ->
                trace Critical (fun () -> $"buildFile / error: {ex |> serializeObj}") _locals
                None, []
            | _ -> None, []
        )

    if fileDir |> SpiralSm.starts_with (repositoryRoot </> "target") then
        let fileDirUri = fileDir |> SpiralFileSystem.normalize_path |> SpiralFileSystem.new_file_uri
        let fileDeleteObj = {| FileDelete = {| uris = [| fileDirUri |] |} |}
        let! _fileDeleteResult = fileDeleteObj |> sendObj serverPort
        ()

    return result
}

## persistCode

In [ ]:
let inline persistCode code = async {
    let targetDir = repositoryRoot </> "target/polyglot/spiral_eval"

    let packagesDir = targetDir </> "packages"

    let hashHex = code |> SpiralCrypto.hash_text

    let codeDir = packagesDir </> hashHex

    codeDir |> System.IO.Directory.CreateDirectory |> ignore

    let mainPath = codeDir </> "main.spi"
    do! code |> SpiralFileSystem.write_all_text_exists mainPath

    let spiprojPath = codeDir </> "package.spiproj"
    let spiprojCode =
        $"""packageDir: {repositoryRoot </> "lib"}
packages:
    |core-
    spiral-
modules:
    main
"""
    do! spiprojCode |> SpiralFileSystem.write_all_text_exists spiprojPath

    return mainPath
}

## buildCode

In [ ]:
let inline buildCode timeout cancellationToken code = async {
    let! mainPath = persistCode code
    let port = getCompilerPort ()
    return! mainPath |> buildFile timeout port cancellationToken
}

In [ ]:
//// test

let buildCode timeout cancellationToken code = buildCode timeout cancellationToken code

In [ ]:
//// test

"""inl app () =
    console.write_line "text"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> buildCode 15000 None
|> Async.runWithTimeout 15000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        Some """let rec closure0 () () : int32 =
    let v0 : string = "text"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""",
        []
    )
)

00:00:22   debug #1 run_with_timeout_async / timeout: 60
00:00:18   debug #1 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:00:19 verbose #2 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:00:19 verbose #3 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:19 verbose #4 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:00:20 verbose #5 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:17 verbose #1 sendJson / port: 13805 / json: {"Ping":true} / result:
00:00:17 verbose #2 awaitCompiler / Ping / result: 'Some(null)' / port: 13

In [ ]:
//// test

""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> _assertEqual None

00:00:34   debug #4 run_with_timeout_async / timeout: 60
00:00:30   debug #8 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:00:30 verbose #9 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:00:30 verbose #10 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:30 verbose #11 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:00:31 verbose #12 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:27 verbose #15 sendJson / port: 13805 / json: {"Ping":true} / result:
00:00:27 verbose #16 awaitCompiler / Ping / result: 'Some(null)' / por

In [ ]:
//// test

"inl app () =
    0i32

inl a = 1

inl main () =
    app
    |> dyn
    |> ignore
"
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "main.spi:
Global inl/let statements should all return functions known at parse time." ]
    )
)

00:00:47   debug #7 run_with_timeout_async / timeout: 60
00:00:43   debug #17 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:00:43 verbose #18 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:00:43 verbose #19 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:43 verbose #20 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:00:44 verbose #21 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:40 verbose #31 sendJson / port: 13805 / json: {"Ping":true} / result:
00:00:40 verbose #32 awaitCompiler / Ping / result: 'Some(null)' / p

In [ ]:
//// test

"""inl main () =
    1i32 / 0i32
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "An attempt to divide by zero has been detected at compile time." ]
    )
)

00:00:53   debug #10 run_with_timeout_async / timeout: 60
00:00:49   debug #24 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:00:50 verbose #25 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:00:50 verbose #26 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:50 verbose #27 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:00:50 verbose #28 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:46 verbose #41 sendJson / port: 13805 / json: {"Ping":true} / result:
00:00:46 verbose #42 awaitCompiler / Ping / result: 'Some(null)' / 

In [ ]:
//// test

"""inl main () =
    1 + ""
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [
            "main.spi:
Constraint satisfaction error.
Got: string
Fails to satisfy: number"
        ]
    )
)

00:01:00   debug #13 run_with_timeout_async / timeout: 60
00:00:56   debug #31 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:00:57 verbose #32 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:00:57 verbose #33 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:57 verbose #34 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:00:57 verbose #35 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:53 verbose #53 sendJson / port: 13805 / json: {"Ping":true} / result:
00:00:53 verbose #54 awaitCompiler / Ping / result: 'Some(null)' / 

In [ ]:
//// test

"""inl main () =
    x + y
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [
            "main.spi:
Unbound variable: x.
Unbound variable: y."
        ]
    )
)

00:01:08   debug #16 run_with_timeout_async / timeout: 60
00:01:04   debug #38 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:01:05 verbose #39 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:01:05 verbose #40 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:05 verbose #41 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:01:05 verbose #42 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:01:01 verbose #67 sendJson / port: 13805 / json: {"Ping":true} / result:
00:01:01 verbose #68 awaitCompiler / Ping / result: 'Some(null)' / 

In [ ]:
//// test

"""union a =
    | B
    | c

inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "main.spi:
Expected: uppercase variable" ]
    )
)

00:01:15   debug #19 run_with_timeout_async / timeout: 60
00:01:11   debug #45 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:01:11 verbose #46 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:01:11 verbose #47 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:11 verbose #48 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:01:12 verbose #49 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:01:08 verbose #79 sendJson / port: 13805 / json: {"Ping":true} / result:
00:01:08 verbose #80 awaitCompiler / Ping / result: 'Some(null)' / 

In [ ]:
//// test

"""
/// abc
inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "main.spi:
Expected: whitespace" ]
    )
)

00:01:20   debug #22 run_with_timeout_async / timeout: 60
00:01:16   debug #52 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:01:16 verbose #53 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:01:16 verbose #54 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:16 verbose #55 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:01:17 verbose #56 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:01:13 verbose #88 sendJson / port: 13805 / json: {"Ping":true} / result:
00:01:13 verbose #89 awaitCompiler / Ping / result: 'Some(null)' / 

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox ()
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "Cannot apply a forall with a term." ]
    )
)

00:01:25   debug #25 run_with_timeout_async / timeout: 60
00:01:21   debug #59 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:01:22 verbose #60 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:01:22 verbose #61 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:22 verbose #62 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:01:22 verbose #63 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:01:19 verbose #98 sendJson / port: 13805 / json: {"Ping":true} / result:
00:01:19 verbose #99 awaitCompiler / Ping / result: 'Some(null)' / 

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox `i32 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        None,
        [ "The main function should not have a forall." ]
    )
)

00:01:34   debug #28 run_with_timeout_async / timeout: 60
00:01:30   debug #66 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:01:31 verbose #67 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:01:31 verbose #68 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:31 verbose #69 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:01:31 verbose #70 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:01:28 verbose #111 sendJson / port: 13805 / json: {"Ping":true} / result:
00:01:28 verbose #112 awaitCompiler / Ping / result: 'Some(null)' 

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.1 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        Some "0.3325000000000001\n",
        []
    )
)

00:01:42   debug #31 run_with_timeout_async / timeout: 60
00:01:38   debug #73 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:01:38 verbose #74 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:01:38 verbose #75 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:38 verbose #76 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:01:38 verbose #77 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:01:35 verbose #123 sendJson / port: 13805 / json: {"Ping":true} / result:
00:01:35 verbose #124 awaitCompiler / Ping / result: 'Some(null)' 

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.01 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _assertEqual (
    Some (
        Some "0.33332500000000004\n",
        []
    )
)
// |> _assertEqual None
// |> fun x -> printfn $"{x.ToDisplayString ()}"

00:01:49   debug #34 run_with_timeout_async / timeout: 60
00:01:44   debug #80 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:buildCode@3-348>, Some "C:\home\git\polyglot")
00:01:45 verbose #81 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:01:45 verbose #82 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:45 verbose #83 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:01:45 verbose #84 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:01:41 verbose #135 sendJson / port: 13805 / json: {"Ping":true} / result:
00:01:41 verbose #136 awaitCompiler / Ping / result: 'Some(null)' 

## getFileTokenRange

In [ ]:
let inline getFileTokenRange port cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let! code = fullPath |> SpiralFileSystem.read_all_text_async
    let lines = code |> SpiralSm.split "\n"

    let struct (token, disposable) = SpiralThreading.new_disposable_token cancellationToken
    use _ = disposable

    let! serverPort, _errors, ct, disposable =
        match port with
        | Some port -> awaitCompiler port (Some token)
        | None -> (getCompilerPort (), FSharp.Control.AsyncSeq.empty, token, new_disposable id) |> Async.init
    use _ = disposable

    let fullPathUri = fullPath |> SpiralFileSystem.normalize_path |> SpiralFileSystem.new_file_uri

    let fileOpenObj = {| FileOpen = {| uri = fullPathUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    // do! Async.Sleep 60

    let fileTokenRangeObj =
        {|
            FileTokenRange =
                {|
                    uri = fullPathUri
                    range =
                        [|
                            {| line = 0; character = 0 |}
                            {| line = lines.Length - 1; character = lines.[lines.Length - 1].Length |}
                        |]
                |}
        |}
    let! fileTokenRangeResult =
        fileTokenRangeObj
        |> sendObj serverPort
        |> Async.withCancellationToken ct

    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    if fileDir |> SpiralSm.starts_with (repositoryRoot </> "target") then
        let fileDirUri = fileDir |> SpiralFileSystem.normalize_path |> SpiralFileSystem.new_file_uri
        let fileDeleteObj = {| FileDelete = {| uris = [| fileDirUri |] |} |}
        let! _fileDeleteResult = fileDeleteObj |> sendObj serverPort
        ()

    return fileTokenRangeResult |> Option.map FSharp.Json.Json.deserialize<int array>
}

## getCodeTokenRange

In [ ]:
let inline getCodeTokenRange cancellationToken code = async {
    let! mainPath = persistCode code
    let port = getCompilerPort ()
    return! mainPath |> getFileTokenRange (Some port) cancellationToken
}

In [ ]:
//// test

"""inl main () = ()"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _assertEqual (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 8; 0; 0; 1; 1; 8; 0 |])

00:02:01   debug #37 run_with_timeout_async / timeout: 60
00:01:57   debug #87 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:it@4-199>, Some "C:\home\git\polyglot")
00:01:57 verbose #88 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:01:57 verbose #89 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:57 verbose #90 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:01:57 verbose #91 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:01:54 verbose #147 sendJson / port: 13805 / json: {"Ping":true} / result:
00:01:54 verbose #148 awaitCompiler / Ping / result: 'Some(null)' / port:

In [ ]:
//// test

"""inl main () = 1i32"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _assertEqual (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 3; 0; 0; 1; 3; 12; 0 |])

00:02:08   debug #40 run_with_timeout_async / timeout: 60
00:02:03   debug #93 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        [||], Some <fun:it@4-376>, Some "C:\home\git\polyglot")
00:02:04 verbose #94 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:02:04 verbose #95 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:02:04 verbose #96 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:02:05 verbose #97 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:02:01 verbose #152 sendJson / port: 13805 / json: {"Ping":true} / result:
00:02:01 verbose #153 awaitCompiler / Ping / result: 'Some(null)' / port:

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | Build_File of string * string
    | File_Token_Range of string * string
    | Execute_Command of string
    | [<Argu.ArguAttributes.Unique>] Timeout of int
    | [<Argu.ArguAttributes.Unique>] Port of int
    | [<Argu.ArguAttributes.Unique>] Parallel

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | Build_File _ -> nameof Build_File
            | File_Token_Range _ -> nameof File_Token_Range
            | Execute_Command _ -> nameof Execute_Command
            | Timeout _ -> nameof Timeout
            | Port _ -> nameof Port
            | Parallel -> nameof Parallel

In [ ]:
//// test

Argu.ArgumentParser.Create<Arguments>().PrintUsage ()

"USAGE: dotnet-repl [--help] [--build-file <string> <string>]
                   [--file-token-range <string> <string>]
                   [--execute-command <string>] [--timeout <int>] [--port <int>]
                   [--parallel]

OPTIONS:

    --build-file <string> <string>
                          Build_File
    --file-token-range <string> <string>
                          File_Token_Range
    --execute-command <string>
                          Execute_Command
    --timeout <int>       Timeout
    --port <int>          Port
    --parallel            Parallel
    --help                display this list of options.
"


## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let buildFileActions =
        argsMap
        |> Map.tryFind (nameof Arguments.Build_File)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.Build_File (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let fileTokenRangeActions =
        argsMap
        |> Map.tryFind (nameof Arguments.File_Token_Range)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.File_Token_Range (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let executeCommandActions =
        argsMap
        |> Map.tryFind (nameof Arguments.Execute_Command)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.Execute_Command command -> Some command
            | _ -> None
        )

    let timeout =
        match argsMap |> Map.tryFind (nameof Arguments.Timeout) with
        | Some [ Arguments.Timeout timeout ] -> timeout
        | _ -> 60000 * 60

    let port =
        match argsMap |> Map.tryFind (nameof Arguments.Port) with
        | Some [ Arguments.Port port ] -> Some port
        | _ -> None

    let isParallel = argsMap |> Map.containsKey (nameof Arguments.Parallel)

    async {
        let port = port |> Option.defaultWith getCompilerPort
        let struct (localToken, disposable) = SpiralThreading.new_disposable_token None
        let! serverPort, _errors, compilerToken, disposable = awaitCompiler port (Some localToken)
        use _ = disposable

        let buildFileAsync =
            buildFileActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let! outputCode, errors = inputPath |> buildFile timeout serverPort None

                errors
                |> List.map snd
                |> List.iter (fun error ->
                    trace Critical (fun () -> $"main / error: {error |> serializeObj}") _locals
                )

                match outputCode with
                | Some outputCode ->
                    do! outputCode |> SpiralFileSystem.write_all_text_async outputPath
                    return 0
                | None ->
                    return 1
            })

        let fileTokenRangeAsync =
            fileTokenRangeActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let! tokenRange = inputPath |> getFileTokenRange (Some serverPort) None
                match tokenRange with
                | Some tokenRange ->
                    do! tokenRange |> FSharp.Json.Json.serialize |> SpiralFileSystem.write_all_text_async outputPath
                    return 0
                | None ->
                    return 1
            })

        let executeCommandAsync =
            executeCommandActions
            |> List.map (fun command -> async {
                let! exitCode, result =
                    SpiralRuntime.execution_options (fun x ->
                        { x with
                            l0 = Some compilerToken
                            l1 = command
                        }
                    )
                    |> SpiralRuntime.execute_with_options_async

                trace Debug (fun () -> $"main / executeCommand / exitCode: {exitCode}") _locals

                return exitCode
            })

        return!
            [| buildFileAsync; fileTokenRangeAsync; executeCommandAsync |]
            |> Seq.collect id
            |> fun x ->
                if isParallel
                then Async.Parallel (x, float System.Environment.ProcessorCount * 0.51 |> ceil |> int)
                else Async.Sequential x
            |> Async.map Array.sum
    }
    |> Async.runWithTimeout timeout
    |> Option.defaultValue 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> SpiralRuntime.split_args
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0